# Male Female Analysis

In [23]:
library(tidyverse)
library(data.table)
library(zoo)
library(tableone)
library(survival)
library(lmerTest)
library(metafor)
library(optimx)
library(broom)

In [51]:
VARS = c("STUDY_NAME", "ID", "TSTART", "DOPA", "AGONIST",'Race', 'LED', 'LDD', 'DAD', 'WGTBL', 'HTBL', 'BMI',
         "Hyposmia", "Cognitive_Impairment","Wearing_Off", "Dyskinesia", "Depression", 
         "RLS","Constipation", "pRBD", "Daytime_Sleepiness", "Insomnia", "HY", 
         "UPDRS1", "UPDRS2","UPDRS3", "UPDRS4", "oldUPDRS","MDS_UPDRS", "MMSE", "MoCA", "SEADL",'UPSIT', 'SEADL70',
         "AGEatBL", "FEMALE", "YEARSEDUC", "BLDfDIAG","AAO","AD", 'FUY')
VARSb = c('Female', "Hyposmia","Cognitive_Impairment", "Wearing_Off", 
          "Dyskinesia", "Depression", "RLS","Constipation", "pRBD", 
          "Daytime_Sleepiness", "Insomnia", "SEADL70")
VARSu = c("UPDRS_scaled", "UPDRS1_scaled", "UPDRS2_scaled", "UPDRS3_scaled", "UPDRS4_scaled")
VARSc = c(VARSu, 'HY', "MMSE", "MoCA", "SEADL")
DENOVOs=c('PPMI', 'PreCEPT_PostCEPT', 'PARKWEST', 'DATATOP')
STUDYs = c(DENOVOs, 'PICNICS', 'NET_PD_LS1', 'DIGPD',  "PDBP", "HBS", "PARKFIT", "PROPARK", "UDALL_PENN")

d = lapply(STUDYs, function(x){fread(sprintf('PDcohorts/%s/standardized.csv', x)) %>% mutate(ID=as.character(ID))}) %>% 
  bind_rows() %>% 
  mutate(STUDY_NAME = ifelse(grepl('PDBP_', STUDY_NAME), 'PDBP', STUDY_NAME)) %>%
  mutate(Age = AGEatBL + TSTART/365.25) %>%
  mutate(DiseaseDuration = BLDfDIAG + TSTART/365.25)

# recode and rename
d = d %>% 
    # set BMI but BMI >50 are highly likely to be scaling mistakes
    mutate(BMI = WGTBL/HTBL/HTBL*100*100) %>%
    mutate(BMI = ifelse(BMI>55, NA, BMI),
           WGTBL = ifelse(BMI>55, NA, WGTBL),
           HTBL = ifelse(BMI>55, NA, HTBL))%>%
    # New hyposmia threshold using UPSIT norm
    mutate(Hyposmia = case_when(
        (FEMALE==1) & (Age<25) & (UPSIT<=35) ~ 1,
        (FEMALE==1) & (Age<50) & (UPSIT<=34) ~ 1,
        (FEMALE==1) & (Age<60) & (UPSIT<=32) ~ 1,
        (FEMALE==1) & (Age<65) & (UPSIT<=31) ~ 1,
        (FEMALE==1) & (Age<70) & (UPSIT<=26) ~ 1,
        (FEMALE==1) & (Age<75) & (UPSIT<=22) ~ 1,
        (FEMALE==1) & (Age<80) & (UPSIT<=17) ~ 1,
        (FEMALE==1) & (Age>=80) & (UPSIT<=15) ~ 1,
        (FEMALE==0) & (Age<40) & (UPSIT<=33) ~ 1,
        (FEMALE==0) & (Age<50) & (UPSIT<=32) ~ 1,
        (FEMALE==0) & (Age<55) & (UPSIT<=29) ~ 1,
        (FEMALE==0) & (Age<65) & (UPSIT<=26) ~ 1,
        (FEMALE==0) & (Age<70) & (UPSIT<=22) ~ 1,
        (FEMALE==0) & (Age<75) & (UPSIT<=19) ~ 1,
        (FEMALE==0) & (Age<80) & (UPSIT<=18) ~ 1,
        (FEMALE==0) & (Age<85) & (UPSIT<=12) ~ 1,
        (FEMALE==0) & (Age>=85) & (UPSIT<=10) ~ 1,
        !is.na(UPSIT) ~ 0)) %>%
#     mutate(Anosmia = (UPSIT<18)*1) %>% 
    rename(Wearing_Off = MOTORFLUX,
          Cognitive_Impairment = MCI,
          Dementia=DEMENTIA,
          MoCA = MOCA,
          Dyskinesia = DYSKINESIAS,
          Depression = DEPR,
          RLS = RL, 
          Constipation = CONST,
          pRBD = RBD,
          Daytime_Sleepiness = SLEEP,
          Insomnia = INS)

db = d %>% filter(TSTART==0) %>% filter(RECRUIT=='PD', DX=='PD') # Only PDs
db %>% with(table(STUDY_NAME, FUY>0))
db = db %>% filter(FUY>0)  # Filter out FUY==0

# Keep the FU>0, and Standardization of UPDRS
d = semi_join(d, db, by = c('STUDY_NAME', 'ID')) # Only keep participants in db
t = db %>% group_by(STUDY_NAME) %>% 
  summarise_at(vars('UPDRS1', 'UPDRS2', 'UPDRS3', 'oldUPDRS', 'MDS_UPDRS'),
               list(~mean(., na.rm = T), ~sd(., na.rm=T))) %>% data.frame
d = left_join(d, t, by = 'STUDY_NAME') %>% 
  mutate(UPDRS1_scaled = ifelse(is.na(UPDRS1_sd), NA, (UPDRS1 - UPDRS1_mean)/UPDRS1_sd),
         UPDRS2_scaled = ifelse(is.na(UPDRS2_sd), NA, (UPDRS2 - UPDRS2_mean)/UPDRS2_sd),
         UPDRS3_scaled = ifelse(is.na(UPDRS3_sd), NA, (UPDRS3 - UPDRS3_mean)/UPDRS3_sd),
         UPDRS4_scaled = scale(UPDRS4),
         UPDRS_scaled = case_when(
           !is.na(MDS_UPDRS_sd) ~ (MDS_UPDRS - MDS_UPDRS_mean)/MDS_UPDRS_sd,
           !is.na(oldUPDRS_sd) ~ (oldUPDRS - oldUPDRS_mean)/oldUPDRS_sd)) %>% 
  select(all_of(unique(c(VARS, VARSu, 'Age', 'DiseaseDuration'))))

# Reset the data at baseline
db = d %>% filter(TSTART==0)

                  
STUDY_NAME         FALSE TRUE
  DATATOP              4  796
  DIGPD               76  350
  HBS                 98  482
  NET_PD_LS1          36 1705
  PARKFIT            120  466
  PARKWEST             6  181
  PDBP               446  486
  PICNICS             11  122
  PPMI                52  408
  PROPARK              6  327
  PreCEPT_PostCEPT     0  390
  UDALL_PENN          19  233

Only include the people with at least one follow-ups. 

In [52]:
d%>% summary

  STUDY_NAME             ID                TSTART            DOPA       
 Length:37852       Length:37852       Min.   :   0.0   Min.   :0.0000  
 Class :character   Class :character   1st Qu.: 101.0   1st Qu.:0.0000  
 Mode  :character   Mode  :character   Median : 414.5   Median :1.0000  
                                       Mean   : 663.5   Mean   :0.5287  
                                       3rd Qu.:1080.0   3rd Qu.:1.0000  
                                       Max.   :3318.0   Max.   :1.0000  
                                                        NA's   :1616    
    AGONIST           Race                LED              LDD        
 Min.   :0.0000   Length:37852       Min.   :   0.0   Min.   :   0.0  
 1st Qu.:0.0000   Class :character   1st Qu.: 300.0   1st Qu.:   0.0  
 Median :0.0000   Mode  :character   Median : 475.0   Median : 300.0  
 Mean   :0.4411                      Mean   : 559.1   Mean   : 370.0  
 3rd Qu.:1.0000                      3rd Qu.: 750.0   3rd Qu.

In [53]:
fwrite(d, 'analysisSet.csv')

# Cohort level analysis

In [54]:
baselineAnalysis = function(df, outcome, covs){
  cat('\n', outcome, 'done for ')
  df[,'Y'] = df[outcome]
  DF = data.frame(STUDY = STUDYs, OUTCOME = outcome, 
                  BETA=NA, SE=NA, Z=NA, P=NA, N=NA, Ncase=NA, 
                  COVS=NA)
  for (STUDY in STUDYs){
    dfs = df %>% filter(STUDY_NAME==STUDY) %>% filter(!is.na(Y))
    if(nrow(dfs)==0){next} # No outcome
    covsNA = colSums(is.na(dfs[covs]), na.rm = T)
    covsNAratio = covsNA/nrow(dfs)
    covsToUse = covs[covsNAratio<0.1]
    dfs = dfs %>% select(all_of(c('ID', 'Y', covsToUse))) %>% 
      filter(complete.cases(.))
    N = nrow(dfs)
    # Analysis
    mod = sprintf('Y ~ %s + I(DiseaseDuration^2) + I(Age^2)', paste(covsToUse, collapse = ' + '))
    if(length(setdiff(dfs$Y, c(0, 1)))==0){ # binomial outcome
      E = sum(dfs$Y) # Number of Cases
      if(E< 25){
          cat(' :SKIP-', STUDY)
          next} # Too few event
      res = glm(mod, data=dfs, family=binomial())
    }else{ # 
      E = NA # No cases
      res = glm(mod, data=dfs)
    }
    resM = summary(res) %>% coef %>% as.matrix
    DF[DF$STUDY==STUDY, 3:6] = resM['FEMALE',]
    DF[DF$STUDY==STUDY, 7:8] = c(N, E)
    DF[DF$STUDY==STUDY, 9] = mod
    cat(STUDY, ', ')
  }
  return(DF)
}
COVs = c('FEMALE', 'Age', 'DiseaseDuration', 'DOPA', 'AGONIST')
result = lapply(c(VARSb[-1], VARSc), function(x){baselineAnalysis(db, x, COVs)})
result = suppressWarnings(bind_rows(result))
fwrite(result, 'result/cohortLevelBL.csv')


 Hyposmia done for PPMI , PDBP , UDALL_PENN , 
 Cognitive_Impairment done for  :SKIP- PPMI :SKIP- PreCEPT_PostCEPTPARKWEST , DATATOP ,  :SKIP- PICNICSNET_PD_LS1 , PDBP , HBS , PARKFIT , PROPARK ,  :SKIP- UDALL_PENN
 Wearing_Off done for  :SKIP- PPMI :SKIP- PARKWEST :SKIP- PICNICSNET_PD_LS1 , DIGPD , PDBP , HBS , PROPARK , UDALL_PENN , 
 Dyskinesia done for  :SKIP- PPMI :SKIP- PARKWEST :SKIP- DATATOP :SKIP- PICNICSNET_PD_LS1 ,  :SKIP- DIGPDPDBP , HBS , PROPARK , UDALL_PENN , 
 Depression done for PPMI , PreCEPT_PostCEPT , PARKWEST ,  :SKIP- DATATOPPICNICS , NET_PD_LS1 , DIGPD , PDBP , HBS , PROPARK , UDALL_PENN , 
 RLS done for PPMI , PARKWEST , DIGPD , PDBP , HBS , 
 Constipation done for PPMI , PARKWEST ,  :SKIP- DATATOPPICNICS , DIGPD , PDBP , PROPARK , 
 pRBD done for PPMI ,  :SKIP- PARKWESTPDBP , 
 Daytime_Sleepiness done for PPMI , PARKWEST ,  :SKIP- DATATOP :SKIP- PICNICSDIGPD , PDBP , PROPARK , 
 Insomnia done for PPMI , PARKWEST , DATATOP , PICNICS , DIGPD , PDBP , HBS , PROPA

In [55]:
coxAnalysis = function(df, outcome, covs){
  cat('\n', outcome, 'done for ')
  DF = data.frame(STUDY = STUDYs, OUTCOME = outcome, 
                  BETA=NA, HR=NA, SE=NA, Z=NA, P=NA, N=NA, E=NA, 
                  COVS=NA)
  df[, 'Y'] = df[outcome]
  for (STUDY in STUDYs){
    dfs = df %>% 
      filter(STUDY_NAME == STUDY) %>%
      filter(!is.na(Y)) %>%
      select(c('ID', 'TSTART', 'Y', 'BLDfDIAG', covs))
    if(nrow(dfs)==0){next} # No outcome
    # Create dataset for survival
    time2follow = dfs %>%
      arrange(TSTART) %>%
      group_by(ID, .keep_all = T) %>%
      mutate(BEGIN = first(TSTART),
             END = last(TSTART)) %>% # BEGIN and END are identical within each ID
      mutate(END = ifelse(Y==1, TSTART, END)) %>% # If 1, then END will be updated
      data.frame %>%
      arrange(END) %>% 
      distinct(ID, .keep_all = T) %>%  # take the smallest END
      select(ID, BEGIN, END) %>% 
      filter(END>0) # Some binomial outcomes only recoreded at baseline > remove
    cohort_by_first_event = left_join(dfs, time2follow, by = 'ID') %>%
      filter(BEGIN <= TSTART & TSTART <= END) %>% # filter observation between BEGIN and END
      arrange(TSTART) %>% 
      group_by(ID) %>% 
      mutate_at(vars(c('Y', covs)), list(~na.locf(., na.rm = F))) %>% # NAs replaced by locf
      ungroup()
    dfsurv = cohort_by_first_event %>% 
      arrange(TSTART) %>% group_by(ID) %>%
      mutate(BLDfDIAG = BLDfDIAG + first(TSTART)/365.25, # Follow-up starts TSTART > 0 sometimes, Needs update
             TSTART = (TSTART - first(TSTART))/365.25) %>% # New baseline set 0 (Especially, ParkWest)
      mutate(TSTOP = lead(TSTART), # The TSTART[i+1] will be the stop time
             event = lead(Y), # the OUTCOME is updated by the one at the stop time
             BLDfDIAGsq = BLDfDIAG^2) %>% #BLDfDIAGsq is updated 
      mutate(TSTOP = ifelse(is.na(TSTOP), END/365.25, TSTOP)) %>% # replace the TSOP with ENNDING TSTART
      data.frame %>% arrange(ID, TSTART) %>% 
      filter(!is.na(event)) # the basially last follow-up data will be erased (Right censored)
    if(nrow(dfsurv)==0){next} # No effective obs
    covsNA = colSums(is.na(dfsurv[covs]))
    covsNAratio = covsNA/nrow(dfsurv)
    covsToUse = covs[covsNAratio<0.1]
    dfsurv = dfsurv[complete.cases(dfsurv[covsToUse]), ]
    N = dfsurv$ID %>% unique %>% length
    E = sum(dfsurv$event)
    if(E<20){cat(STUDY);next} # Less than 20 outcomes
    mod = sprintf('Surv(TSTART, TSTOP, event == 1) ~ %s', paste(covsToUse, collapse = ' + '))
    mod = sprintf('Surv(TSTART, TSTOP, event == 1) ~ %s + I(Age^2) + I(DiseaseDuration^2)', paste(covsToUse, collapse = ' + '))
    res = coxph(as.formula(mod), data=dfsurv)
    resM = summary(res) %>% coef %>% as.matrix 
    DF[DF$STUDY==STUDY, 3:7] = resM['FEMALE',]
    DF[DF$STUDY==STUDY, 8:9] = c(N, E)
    DF[DF$STUDY==STUDY, 10]  = mod
    cat(STUDY,', ')
  }
  return(DF)
}
COVs = c('FEMALE', 'Age', 'DiseaseDuration', 'DOPA', 'AGONIST')
result = lapply(VARSb[-1], function(x){coxAnalysis(d, x, COVs)})
result = suppressWarnings(bind_rows(result))
fwrite(result, 'result/cohortLevelSurv.csv') 


 Hyposmia done for 

Warning message in fitter(X, Y, istrat, offset, init, control, weights = weights, :
"Ran out of iterations and did not converge"


PPMI , PreCEPT_PostCEPT , PDBP , UDALL_PENN
 Cognitive_Impairment done for PPMI , PreCEPT_PostCEPT , PARKWEST , DATATOP , PICNICS , NET_PD_LS1 , PDBP , HBS , PROPARK , UDALL_PENN , 
 Wearing_Off done for PPMI , PreCEPT_PostCEPT , PARKWEST , PICNICS , NET_PD_LS1 , DIGPD , PDBP , HBS , PROPARK , UDALL_PENN , 
 Dyskinesia done for PPMI , PreCEPT_PostCEPT , PARKWEST , DATATOPPICNICSNET_PD_LS1 , DIGPD , PDBP , HBS , PROPARK , UDALL_PENN , 
 Depression done for PPMI , PreCEPT_PostCEPT , PARKWEST , DATATOP , PICNICS , NET_PD_LS1 , DIGPD , PDBP , HBS , PROPARK , UDALL_PENN , 
 RLS done for PPMI , PARKWEST , DIGPD , PDBP , HBS , PROPARK
 Constipation done for PPMI , PreCEPT_PostCEPT , PARKWEST , DATATOPPICNICS , DIGPD , PDBP , PROPARK , 
 pRBD done for PPMI , PARKWEST , PDBP , 
 Daytime_Sleepiness done for PPMI , PreCEPT_PostCEPT , PARKWEST , DATATOPPICNICS , DIGPD , PDBP , PROPARK , 
 Insomnia done for PPMI , PreCEPT_PostCEPT , PARKWEST , DATATOP , PICNICS , DIGPD , PDBP , HBS , PROPARK , 
 SE

In [56]:
lmerAnalysis = function(df, outcome, covs){
  cat('\n', outcome, 'done for ')
  df[,'Y'] = df[,outcome] # sometimes df[outcome] works instead of df[,outcome]
  DF = data.frame(STUDY = STUDYs, OUTCOME = outcome, 
                  BETA=NA, SE=NA, DF=NA, Tval=NA, P=NA, N=NA, N_obs=NA, 
                  COVS=NA)
  for (STUDY in STUDYs){
    dfs = df %>% filter(STUDY_NAME==STUDY) %>% 
      filter(!is.na(Y)) %>% 
      mutate_at(vars(covs), list(~na.locf(., na.rm=F))) %>% 
      mutate(TSTART=TSTART/365.25) %>%
      group_by(ID) %>%
      filter(n()>=2) %>%
      ungroup()
    if(nrow(dfs)==0){next} # No outcome
    covsNA = colSums(is.na(dfs[covs]))
    covsNAratio = covsNA/nrow(dfs)
    covsToUse = covs[covsNAratio<0.1]
    dfs = dfs[complete.cases(dfs[covsToUse]), ] # Complete set
    covsOneValue = covsToUse[apply(dfs[covsToUse], 2, sd)==0] # Variables only have one value
    covsToUseNoFemale = setdiff(covsToUse, c('FEMALE', covsOneValue))
      
    # reslacing
    scale1 <- function(x){scale(x)[,1]}
    dfs = dfs %>%
        mutate_at(vars(all_of(covsToUseNoFemale)), scale1)
    N_obs = nrow(dfs)
    
    N = length(unique(dfs$ID))
    if(N_obs<3*N){next} # follow-up too few to accomodate random slope
    # Analysis
#     mod = sprintf('Y ~ %s + FEMALE:TSTART + (TSTART|ID) + I(TSTART^2)', paste(covsToUse, collapse = ' + '))
#     mod = sprintf('Y ~ %s + FEMALE:TSTART + (TSTART|ID) + I(TSTART^2) + I(AGEatBL^2)', paste(covsToUse, collapse = ' + '))
#     mod = sprintf('Y ~ %s + FEMALE:TSTART + (DiseaseDuration|ID) + I(Age^2) + I(DiseaseDuration^2)', paste(covsToUseNoFemale, collapse = ' + '))
    mod = sprintf('Y ~ %s + FEMALE + FEMALE:DiseaseDuration + (DiseaseDuration|ID) + I(Age^2) + I(DiseaseDuration^2)', paste(covsToUseNoFemale, collapse = ' + '))
#     mod = sprintf('Y ~ %s + FEMALE*TSTART + (DiseaseDuration|ID) + I(Age^2) + I(DiseaseDuration^2)', paste(covsToUseNoFemale, collapse = ' + '))
    res = lmer(as.formula(mod), data=dfs, control = lmerControl(optimizer = "optimx", optCtrl = list(method = "nlminb")))
    resM = summary(res) %>% coef %>% as.matrix
#     DF[DF$STUDY==STUDY, 3:7] = resM['FEMALE:TSTART',]
    DF[DF$STUDY==STUDY, 3:7] = resM['DiseaseDuration:FEMALE',]
    DF[DF$STUDY==STUDY, 8:9] = c(N, N_obs)
    DF[DF$STUDY==STUDY, 10]  = mod
    cat(STUDY, ', ')
  }
  return(DF)
}
COVs = c('FEMALE', 'Age', 'DiseaseDuration', 'DOPA', 'AGONIST')
result = lapply(VARSc, function(x){lmerAnalysis(d, x, COVs)})
result = suppressWarnings(bind_rows(result))
fwrite(result, 'result/cohortLevelMIX.csv')


 UPDRS_scaled done for PPMI , PreCEPT_PostCEPT , PARKWEST , DATATOP , PICNICS , NET_PD_LS1 , DIGPD , PDBP , 

boundary (singular) fit: see ?isSingular



UDALL_PENN , 
 UPDRS1_scaled done for PPMI , PreCEPT_PostCEPT , PARKWEST , NET_PD_LS1 , DIGPD , PDBP , 

boundary (singular) fit: see ?isSingular



UDALL_PENN , 
 UPDRS2_scaled done for PPMI , PreCEPT_PostCEPT , PARKWEST , NET_PD_LS1 , DIGPD , PDBP , UDALL_PENN , 
 UPDRS3_scaled done for PPMI , PreCEPT_PostCEPT , PARKWEST , NET_PD_LS1 , DIGPD , PDBP , UDALL_PENN , 
 UPDRS4_scaled done for 

...

PDBP , UDALL_PENN , 
 SEADL done for PPMI , PreCEPT_PostCEPT , PARKWEST , DATATOP , NET_PD_LS1 , DIGPD , 

boundary (singular) fit: see ?isSingular



PDBP , 

boundary (singular) fit: see ?isSingular



UDALL_PENN , 

# Meta-analysis

ParkFit only has 2 time points - baseline and 2 years later. 
-> exclude from cox meta-analysis

In [57]:
createLogisticForest = function(OUTCOMEi, df){
  df_i = df %>% 
    filter(OUTCOME==OUTCOMEi, SE>1E-5)
  n_all = length(df_i$STUDY)
  n_denovo = length(intersect(df_i$STUDY, DENOVOs))
  if(n_all>1){
    if(n_denovo>1){
    k1 = rma(yi=BETA, sei= SE, data= df_i %>% filter(STUDY %in% DENOVOs), method = "REML")
    }else{k1=NULL}
    if(n_all>n_denovo){
    k2 = rma(yi=BETA, sei= SE, data=df_i %>% filter(!(STUDY %in% DENOVOs)), method = "REML", slab=STUDY)
    }else{k2=NULL}
    k3 = rma(yi=BETA, sei= SE, data=df_i, method = "REML", slab=STUDY)
    Ylab = c(n_all + 6 - c(1:n_denovo, (n_denovo+4):(n_all+3)))
    if(n_denovo==0){Ylab = c(n_all + 6 - (n_denovo+4):(n_all+3))}
    op <- par(cex = 1, font=1)
    print(OUTCOMEi)
    figout = sprintf('result/%s_Logistic.png', OUTCOMEi)
    png(figout, width=2000, height=2000, res=300)
    forest(k3, cex=.9, rows = Ylab, ylim = c(-1, max(Ylab) + 4),
         xlim=c(-5, 3), at=log(c(0.25, 0.5, 1, 2, 3, 4)),
         ilab = sprintf('%d / %d', df_i$Ncase, df_i$N-df_i$Ncase),
         ilab.xpos = -2,
         atransf=exp,
         mlab=sprintf('Meta-analysis for all \n(P = %.2g , I_sq = %.1f%%, Q-test = %.3f)', 
                      k3$pval, k3$I2, k3$QEp), 
         addfit=T, xlab = 'Odds Ratio',
         main = paste(OUTCOMEi,': sex differences in odds at baseline\n(reference - Male)'),
         order=order(factor(df_i$STUDY, levels=STUDYs)))
    text(-5, max(Ylab)+2.5, bquote("Cohort"), pos=4, cex=.9)
    text(-2.7, max(Ylab)+2.5, bquote("Case / Control"), pos=4, cex=.8)
    text(3, max(Ylab)+2.5, bquote("OR [95% C.I.]"), pos=2, cex=.9)
    op <- par(cex=.8, font=4)
    if(n_denovo>0){text(-5, max(Ylab)+1, bquote("De novo Cohorts"), pos=4)}
    text(-5,  n_all + 2.8 - n_denovo, "Other Cohorts", pos=4)
    op <- par(cex=0.8, font=3)
    if(n_denovo>1){
    addpoly(k1, atransf=exp, row= n_all + 4.8 - n_denovo, col="grey",
            mlab = sprintf('Meta-analysis for de novo cohorts \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                           k1$pval, k1$I2, k1$QEp))
    }
    if((n_all - n_denovo)>1){
    addpoly(k2, atransf=exp, row= 1.8, col="grey",
            mlab = sprintf('Meta-analysis for the other cohorts \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                           k2$pval, k2$I2, k2$QEp))
    }
    op <- par(cex=1, font=1)
    dev.off() 
    write(paste(OUTCOMEi, 'logistic', k3$beta, k3$se, k3$pval, k3$I2, k3$QEp, sep=','), file = 'result/meta.csv', append=T)
  }
}
write(paste('OUTCOME', 'MODEL', 'BETA', 'SE', 'P', 'Isq', 'QEp', sep=','), file = 'result/meta.csv', append=F)
lapply(VARSb[-1], function(x){createLogisticForest(x, fread('result/cohortLevelBL.csv'))})

[1] "Hyposmia"
[1] "Cognitive_Impairment"
[1] "Wearing_Off"
[1] "Dyskinesia"
[1] "Depression"
[1] "RLS"
[1] "Constipation"
[1] "pRBD"
[1] "Daytime_Sleepiness"
[1] "Insomnia"
[1] "SEADL70"


[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

In [58]:
df =  fread('result/cohortLevelSurv.csv')
for(OUTCOMEi in VARSb[-1]){
#   df_i = df %>% filter(OUTCOME==OUTCOMEi, abs(BETA)<1.5, SE>1E-5) # PARKFIT HY3 SURV
  df_i = df %>% filter(OUTCOME==OUTCOMEi, SE>1E-5) # PARKFIT HY3 SURV
  n_all = length(df_i$STUDY)
  n_denovo = length(intersect(df_i$STUDY, DENOVOs))
  if(n_all>1){
    if(OUTCOMEi=='MCI'){OUTCOMEi='Cognitive_Impairment'}
    print(OUTCOMEi)
    figout = sprintf('result/%s_Surv.png', OUTCOMEi)
    png(figout, width=2000, height=2000, res=300)
    if(n_denovo>1){
    k1 = rma(yi=BETA, sei= SE, data= df_i %>% filter(STUDY %in% DENOVOs), method = "REML")
    }else{k1=NULL}
    if(n_all>n_denovo){
    k2 = rma(yi=BETA, sei= SE, data=df_i %>% filter(!(STUDY %in% DENOVOs)), method = "REML", slab=STUDY)
    }else{k2=NULL}
    k3 = rma(yi=BETA, sei= SE, data=df_i, method = "REML", slab=STUDY)
    Ylab = c(n_all + 6 - c(1:n_denovo, (n_denovo+4):(n_all+3)))
    if(n_denovo==0){Ylab = c(n_all + 6 - (n_denovo+4):(n_all+3))}
    op <- par(cex = 1, font=1)
    forest(k3, cex=.9, rows = Ylab, ylim = c(-1, max(Ylab) + 4),
         xlim=c(-5, 3), at=log(c(0.25, 0.5, 1, 2, 3, 4)),
         ilab = sprintf('%d / %d', df_i$E, df_i$N),
         ilab.xpos = -2,
         atransf=exp,
         mlab=sprintf('Meta-analysis for all \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                      k3$pval, k3$I2, k3$QEp), 
         addfit=T, xlab = 'Hazard Ratio',
         main = paste(OUTCOMEi, ': sex differences in survival\n(reference - Male)'),
         order=order(factor(df_i$STUDY, levels=STUDYs)))
    text(-5, max(Ylab)+2.5, bquote("Cohort"), pos=4, cex=.9)
    text(-2.6, max(Ylab)+2.5, bquote("Event / N"), pos=4, cex=.8)
    text(3, max(Ylab)+2.5, bquote("HR [95% C.I.]"), pos=2, cex=.9)
    op <- par(cex=.8, font=4)
    if(n_denovo>0){text(-5, max(Ylab)+1, bquote("de novo Cohorts"), pos=4)}
    text(-5,  n_all + 2.8 - n_denovo, "Other Cohorts", pos=4)
    op <- par(cex=0.8, font=3)
    if(n_denovo>1){
    addpoly(k1, atransf=exp, row= n_all + 4.8 - n_denovo, col="grey",
            mlab = sprintf('Meta-analysis for de novo cohorts \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                           k1$pval, k1$I2, k1$QEp))
    }
    if((n_all - n_denovo)>1){
    addpoly(k2, atransf=exp, row= 1.8, col="grey",
            mlab = sprintf('Meta-analysis for the other cohorts \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                           k2$pval, k2$I2, k2$QEp))
    }
    op <- par(cex=1, font=1)
    dev.off()
    write(paste(OUTCOMEi, 'survival', k3$beta, k3$se, k3$pval, k3$I2, k3$QEp, sep=','), file = 'result/meta.csv', append=T)
  }
}

[1] "Hyposmia"
[1] "Cognitive_Impairment"
[1] "Wearing_Off"
[1] "Dyskinesia"
[1] "Depression"
[1] "RLS"
[1] "Constipation"
[1] "pRBD"
[1] "Daytime_Sleepiness"
[1] "Insomnia"
[1] "SEADL70"


In [59]:
createLinearForest = function(OUTCOMEi, df){
  df_i = df %>% filter(OUTCOME==OUTCOMEi, !is.na(BETA), SE>1E-5) # No beta 
  n_all = length(df_i$STUDY)
  n_denovo = length(intersect(df_i$STUDY, DENOVOs))
  if(n_all>1){
    print(OUTCOMEi)
    if(n_denovo>1){
      k1 = rma(yi=BETA, sei= SE, data= df_i %>% filter(STUDY %in% DENOVOs), method = "REML")
    }else{k1=NULL}
    if(n_all>n_denovo){
      k2 = rma(yi=BETA, sei= SE, data=df_i %>% filter(!(STUDY %in% DENOVOs)), method = "REML", slab=STUDY)
    }else{k2=NULL}
    
    k3 = rma(yi=BETA, sei= SE, data=df_i, method = "REML", slab=STUDY)
    Ylab = c(n_all + 6 - c(1:n_denovo, (n_denovo+4):(n_all+3)))
    if(n_denovo==0){Ylab = c(n_all + 6 - (n_denovo+4):(n_all+3))}
    op <- par(cex = 1, font=1)
    Bmin = min(0, df_i$BETA)
    Bmax = max(df_i$BETA, 0)
    Blen = Bmax - Bmin
    Xmin = Bmin - 5*Blen
    Xmax = Bmax + 3*Blen
    figout = sprintf('result/%s_Linear.png', OUTCOMEi)
    png(figout, width=2000, height=2000, res=300)
    forest(k3, cex=.9, rows = Ylab, ylim = c(-1, max(Ylab) + 4),
           xlim=c(Xmin, Xmax), at=c(Bmin-Blen, Bmin, Bmax, Bmax+Blen, 0),
           ilab = sprintf('%d', df_i$N),
           ilab.xpos = Bmin-2*Blen,
           mlab=sprintf('Meta-analysis for all \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                        k3$pval, k3$I2, k3$QEp), 
           addfit=T, xlab = 'Difference',
           main = paste(OUTCOMEi, ': sex differences at baseline\n(reference - Male)'),
           order=order(factor(df_i$STUDY, levels=STUDYs)))
    text(Xmin, max(Ylab)+2.5, bquote("Cohort"), pos=4, cex=.9)
    text(Bmin-2.3*Blen, max(Ylab)+2.5, bquote("N"), pos=4, cex=.9)
    text(Xmax, max(Ylab)+2.5, bquote("Mean [95% C.I.]"), pos=2, cex=.9)
    op <- par(cex=.8, font=4)
    if(n_denovo>0){text(Xmin, max(Ylab)+1, bquote("de novo Cohorts"), pos=4)}
    text(Xmin,  n_all + 2.8 - n_denovo, "Other Cohorts", pos=4)
    op <- par(cex=0.8, font=3)
    if(n_denovo>1){
      addpoly(k1, row= n_all + 4.8 - n_denovo, col="grey",
              mlab = sprintf('Meta-analysis for de novo cohorts \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                             k1$pval, k1$I2, k1$QEp))
    }
    if((n_all - n_denovo)>1){
      addpoly(k2, row= 1.8, col="grey",
              mlab = sprintf('Meta-analysis for the other cohorts \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                             k2$pval, k2$I2, k2$QEp))
    }
    op <- par(cex=1, font=1)
    dev.off()
    write(paste(OUTCOMEi, 'linear', k3$beta, k3$se, k3$pval, k3$I2, k3$QEp, sep=','), file = 'result/meta.csv', append=T)
  }
}
lapply(VARSc, function(x){createLinearForest(x, fread('result/cohortLevelBL.csv'))})

[1] "UPDRS_scaled"
[1] "UPDRS1_scaled"
[1] "UPDRS2_scaled"
[1] "UPDRS3_scaled"
[1] "UPDRS4_scaled"
[1] "HY"
[1] "MMSE"
[1] "MoCA"
[1] "SEADL"


[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

In [60]:
df =  fread('result/cohortLevelMIX.csv')
createLinearForest = function(OUTCOMEi, df){
  df_i = df %>% filter(OUTCOME==OUTCOMEi, !is.na(BETA), SE>1E-5) # No beta thres
  n_all = length(df_i$STUDY)
  n_denovo = length(intersect(df_i$STUDY, DENOVOs))
  if(n_all>1){
    print(OUTCOMEi)
    if(n_denovo>1){
      k1 = rma(yi=BETA, sei= SE, data= df_i %>% filter(STUDY %in% DENOVOs), method = "REML")
    }else{k1=NULL}
    if(n_all>n_denovo){
      k2 = rma(yi=BETA, sei= SE, data=df_i %>% filter(!(STUDY %in% DENOVOs)), method = "REML", slab=STUDY)
    }else{k2=NULL}
    
    k3 = rma(yi=BETA, sei= SE, data=df_i, method = "REML", slab=STUDY, digits=5, control=list(stepadj=0.5))
    Ylab = c(n_all + 6 - c(1:n_denovo, (n_denovo+4):(n_all+3)))
    if(n_denovo==0){Ylab = c(n_all + 6 - (n_denovo+4):(n_all+3))}
    op <- par(cex = 1, font=1)
    Bmin = min(0, df_i$BETA)
    Bmax = max(df_i$BETA, 0)
    Blen = Bmax - Bmin
    Xmin = Bmin - 5*Blen
    Xmax = Bmax + 3*Blen
    figout = sprintf('result/%s_Mixed.png', OUTCOMEi)
    png(figout, width=1440, height=1440, res=216)
    forest(k3, cex=.9, rows = Ylab, ylim = c(-1, max(Ylab) + 4),
           xlim=c(Xmin, Xmax), at=c(Bmin-Blen, Bmin, Bmax, Bmax+Blen, 0),
           ilab = sprintf('%d / %d', df_i$N_obs, df_i$N),
           ilab.xpos = Bmin-2*Blen,
           mlab=sprintf('Meta-analysis for all \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                        k3$pval, k3$I2, k3$QEp), 
           addfit=T, xlab = 'Difference',
           main = paste(OUTCOMEi, ': sex differences in rate of change per year\n(reference - Male)'),
           order=order(factor(df_i$STUDY, levels=STUDYs)))
    text(Xmin, max(Ylab)+2.5, bquote("Cohort"), pos=4, cex=.9)
    text(Bmin-2.6*Blen, max(Ylab)+2.5, bquote("N_obs/N"), pos=4, cex=.9)
    text(Xmax, max(Ylab)+2.5, bquote("Mean [95% C.I.]"), pos=2, cex=.9)
    op <- par(cex=.8, font=4)
    if(n_denovo>0){text(Xmin, max(Ylab)+1, bquote("de novo Cohorts"), pos=4)}
    text(Xmin,  n_all + 2.8 - n_denovo, "Other Cohorts", pos=4)
    op <- par(cex=0.8, font=3)
    if(n_denovo>1){
      addpoly(k1, row= n_all + 4.8 - n_denovo, col="grey",
              mlab = sprintf('Meta-analysis for de novo cohorts \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                             k1$pval, k1$I2, k1$QEp))
    }
    if((n_all - n_denovo)>1){
      addpoly(k2, row= 1.8, col="grey",
              mlab = sprintf('Meta-analysis for the other cohorts \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                             k2$pval, k2$I2, k2$QEp))
    }
    op <- par(cex=1, font=1)
    dev.off()
    write(paste(OUTCOMEi, 'mixed', k3$beta, k3$se, k3$pval, k3$I2, k3$QEp, sep=','), file = 'result/meta.csv', append=T)
  }
}
lapply(VARSc, function(x){createLinearForest(x, fread('result/cohortLevelMIX.csv'))})

[1] "UPDRS_scaled"
[1] "UPDRS1_scaled"
[1] "UPDRS2_scaled"
[1] "UPDRS3_scaled"
[1] "UPDRS4_scaled"
[1] "HY"
[1] "MMSE"
[1] "MoCA"
[1] "SEADL"


[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

In [76]:
df = fread('result/meta.csv') %>% filter(!(OUTCOME %in% c('oldUPDRS', 'MDS_UPDRS', 'HY3')))
nrow(df)
df_adj =df %>% 
    mutate(RANK = ifelse(QEp>0.05, 1, NA)) %>% 
    arrange(RANK, P) %>% 
    mutate(RANK = cumsum(RANK)) %>%
    mutate(Pfdr = ifelse(P*RANK<1, P*RANK, 1)) %>%
    mutate(Ntest = max(RANK, na.rm=T)) %>%
    mutate(Pbon = ifelse(is.na(RANK), NA, P*Ntest)) %>%
    mutate(Pbon = ifelse(Pbon>1, 1, Pbon)) %>%
    select(-Ntest) %>%
    mutate(MeanCI = case_when(
           MODEL=='survival' ~ sprintf('%.3f [%.3f, %.3f] (HR)', exp(BETA), exp(BETA - 1.96*SE), exp(BETA + 1.96*SE)),
           MODEL=='logistic' ~ sprintf('%.3f [%.3f, %.3f] (OR)', exp(BETA), exp(BETA - 1.96*SE), exp(BETA + 1.96*SE)),
           TRUE~sprintf('%.3f [%.3f, %.3f]', BETA, BETA - 1.96*SE, BETA + 1.96*SE)))
fwrite(df_adj, 'result/metaPadj.csv')
df_adj %>% 
    arrange(P) %>%
    mutate(MODEL2 = case_when(
        MODEL %in% c('survival', 'mixed') ~ '1_progression',
        TRUE~'2_baseline'),
      MeanCI = case_when(
        MODEL=='survival' ~ sprintf('%.2f [%.2f, %.2f] (HR)', exp(BETA), exp(BETA - 1.96*SE), exp(BETA + 1.96*SE)),
        MODEL=='logistic' ~ sprintf('%.2f [%.2f, %.2f] (OR)', exp(BETA), exp(BETA - 1.96*SE), exp(BETA + 1.96*SE)),
        TRUE~sprintf('%.2f [%.2f, %.2f]', BETA, BETA - 1.96*SE, BETA + 1.96*SE))) %>%
    arrange(MODEL2, OUTCOME) %>%
    select(OUTCOME, BETA, SE, P, Pbon, MeanCI, QEp, MODEL) %>%
    filter(Pbon<0.05) %>%
    fwrite(., 'result/metaPadj_reduced.csv')

[1] 40

# Fox Insight

In [86]:
df = fread('PDcohorts/FoxInsight/rawdata/Processed.csv', na.strings = '')

In [87]:
t = df  %>% 
        distinct(ID, .keep_all=T) %>%
        summarize_at(vars('AgeAtBL', 'BLYfDx'), list(
        m = ~mean(., na.rm = T), 
        Min = ~min(., na.rm=T),
        q1 = ~quantile(.,prob = 0.1, na.rm = T),
        q2 = ~median(., na.rm = T), 
        q3 = ~quantile(.,prob = 0.9, na.rm = T),
        Max = ~max(., na.rm=T)))
t(t)

AgeAtBL_m,64.217369
BLYfDx_m,6.257859
AgeAtBL_Min,20.000000
BLYfDx_Min,0.000000
AgeAtBL_q1,48.600000
BLYfDx_q1,1.000000
AgeAtBL_q2,66.000000
BLYfDx_q2,4.600000
AgeAtBL_q3,77.400000
BLYfDx_q3,13.500000
AgeAtBL_Max,90.000000


In [88]:
dim(df)
df = df %>%
    filter(AgeAtBL > t$AgeAtBL_q1, Age < t$AgeAtBL_q3) %>%
    filter((Recruit=='CTR')|((BLYfDx > t$BLYfDx_q1) & (BLYfDx < t$BLYfDx_q3)))
dim(df)

[1] 165597    128

[1] 119493    128

In [89]:
df %>% 
    distinct(ID, .keep_all=T) %>%
    with(table(Recruit))

Recruit
  CTR    PD 
 7588 19390 

In [90]:
df = df %>% 
    mutate(Recruit = case_when(
        Recruit=='CTR'~ 'CTR',
        DiseaseDuration<3~'PD1',
        DiseaseDuration<=6~'PD2',
        DiseaseDuration>6~'PD3'
    ))
df %>% 
    distinct(ID, .keep_all=T) %>%
    with(table(Recruit))

Recruit
 CTR  PD1  PD2  PD3 
7588 7658 5805 5927 

In [91]:
# Fill in the levodopa and agonoist parameter
df = df %>% 
    arrange(DaysFromIdx) %>%
    group_by(ID) %>%
    mutate_at(vars('LEVODOPA', 'AGONIST'), ~na.locf(., na.rm = F)) %>%
    ungroup()

In [92]:
df$Depression = df$DEPR
df$Wearing_Off = df$WearingOff
Outcomes = c('UPDRS2', 'PDAQ15_total', 'Dyskinesia', 'Wearing_Off', 'Depression', 'pRBD',
             'NMSQ_Awake', 'NMSQ_Sleep', 'NMSQ_Feel', 'NMSQ_Constipation', 'NMSQ_Smell')
BinomOutcomes = Outcomes[-c(1,2)]

In [93]:
fwrite(df, 'analysisSetFI.csv')

In [69]:
# Interaction between disease duration and sex
res = data.frame(outcome = rep(Outcomes),
                n=NA, o=NA, 
                beta_slp=NA, se_slp=NA, pval_slp=NA,
                beta_main=NA, se_main=NA, pval_main=NA)
for (Outcome in Outcomes){
    cat('\n', Outcome)
    df[, 'y'] = df[Outcome]
    t = df %>% 
        filter(Recruit!='CTR') %>%
        filter(!is.na(y)) %>% 
        filter(!is.na(Age)) %>% 
        filter(!is.na(Female)) %>% 
        arrange(DaysFromIdx) %>% 
        distinct(ID, .keep_all = T)
    Fam = 'gaussian()'
    if(Outcome %in% BinomOutcomes){Fam='binomial()'}
    cat(Fam)  
#     mod = glm(y~Female*DiseaseDuration+Age+I(Age^2)+LEVODOPA+AGONIST, data=t, family = eval(parse(text=Fam)))
    mod = glm(y~Female*DiseaseDuration+Age+I(Age^2)+I(DiseaseDuration^2)+LEVODOPA+AGONIST, data=t, family = eval(parse(text=Fam)))
    mPD = coef(summary(mod))
    res[res$outcome==Outcome, 2:9] = c(nrow(t), sum(t$y), 
                                       mPD['Female:DiseaseDuration',-3],
                                       mPD['Female',-3])
} 
fwrite(res, 'result/FoxInsightPD.csv')


 UPDRS2gaussian()
 PDAQ15_totalgaussian()
 Dyskinesiabinomial()
 Wearing_Offbinomial()
 Depressionbinomial()
 pRBDbinomial()
 NMSQ_Awakebinomial()
 NMSQ_Sleepbinomial()
 NMSQ_Feelbinomial()
 NMSQ_Constipationbinomial()
 NMSQ_Smellbinomial()

In [70]:
# Main effect
res = data.frame(outcome = rep(Outcomes),
                n=NA, o=NA,
                beta_main=NA, se_main=NA, pval_main=NA)
for (Outcome in Outcomes){
    cat('\n', Outcome)
    df[, 'y'] = df[,Outcome]
    t = df %>% 
    filter(Recruit=='CTR') %>%
    filter(!is.na(y)) %>% 
    filter(!is.na(Age)) %>% 
    filter(!is.na(Female)) %>% 
    arrange(DaysFromIdx) %>% 
    distinct(ID, .keep_all = T)
    Fam = 'gaussian()'
    if(length(unique(t$y))==2){Fam='binomial()'}
    if(nrow(t)<300){next}
    cat(Fam)  
    mod = glm(y~Female+Age+I(Age^2), data=t, family = eval(parse(text=Fam)))
    mPD = coef(summary(mod))
    res[res$outcome==Outcome, 2:6] = c(nrow(t), sum(t$y), 
                                       mPD['Female',-3])
} 
fwrite(res, 'result/FoxInsightCTR.csv')


 UPDRS2
 PDAQ15_totalgaussian()
 Dyskinesia
 Wearing_Off
 Depressionbinomial()
 pRBDbinomial()
 NMSQ_Awakebinomial()
 NMSQ_Sleepbinomial()
 NMSQ_Feelbinomial()
 NMSQ_Constipationbinomial()
 NMSQ_Smellbinomial()

## MetaAnalysis

In [71]:
# Create stratified analysis
# Interaction between disease duration and sex
res = data.frame(outcome = rep(Outcomes, each=3),
                 strata = rep(c('PD1', 'PD2', 'PD3'), length(Outcomes)),
                n=NA, o=NA, 
                beta_main=NA, se_main=NA, pval_main=NA)
for (Outcome in Outcomes){
    df[,'y'] = df[,Outcome]
    for (Strata in c('PD1', 'PD2', 'PD3')){
        t = df %>% 
            filter(Recruit==Strata) %>%
            filter(!is.na(y)) %>% 
            filter(!is.na(Age)) %>% 
            filter(!is.na(Female)) %>% 
            arrange(DaysFromIdx) %>% 
            distinct(ID, .keep_all = T)
        Fam = 'gaussian()'
        if(length(unique(t$y))==2){Fam='binomial()'}
        mod = glm(y~Female+Age+I(Age^2)+I(DiseaseDuration^2)+LEVODOPA+AGONIST, data=t, family = eval(parse(text=Fam)))
        mPD = coef(summary(mod))
        res[(res$outcome==Outcome)&(res$strata==Strata), 3:7] = c(nrow(t), sum(t$y), mPD['Female',-3])        
    }
}
fwrite(res, 'result/FoxInsightPDstrata.csv')

In [77]:
mPD = fread('result/FoxInsightPD.csv') %>% mutate(strata='PD_all')
mPDstratified = fread('result/FoxInsightPDstrata.csv')
mCTR = fread('result/FoxInsightCTR.csv') %>% mutate(strata='CTR')
res = bind_rows(mPDstratified, mPD, mCTR) %>%
    select(names(mPDstratified)) %>%
    rename(beta=beta_main, se=se_main, pval=pval_main) %>%
    arrange(outcome, strata)

In [78]:
res = res %>% 
    mutate(strata2 = case_when(
        strata=='PD1'~'PD duration 1-3y',
        strata=='PD2'~'PD duration 3-6y',
        strata=='PD3'~'PD duration > 6y',
        TRUE~strata
    ))

In [81]:
meta = data.frame(outcome = Outcomes,strata = 'Main',
                 n=NA, o=NA, beta=NA, se=NA, pval=NA, I2=NA, QEp=NA)

figout = sprintf('result/FI_All.png', Outcome)
png(figout, width=4000, height=12000, res=300)    
par(mfrow=c(6, 2))

for (Outcome in Outcomes){
  res_m = res %>% filter(outcome==Outcome, !is.na(beta))
  print(Outcome)
  if(nrow(res_m)>=2){
#     figout = sprintf('result/FI_%s.png', Outcome)
#     png(figout, width=2000, height=2000, res=300)
    op <- par(cex = 1, font=1)
    Bmin = min(0, res_m$beta - res_m$se)
    Bmax = max(res_m$beta+res_m$se, 0)
    Blen = Bmax - Bmin
    Xmin = Bmin - 5*Blen
    Xmax = Bmax + 3*Blen
    
    mAll = rma(yi=beta, sei= se, data= res_m, method='REML', slab=strata2)
    Allrow = length(res_m$strata)
    Mainrow = sum(res_m$strata %in% c('PD_all','CTR'))
    Ylab = (Allrow-Mainrow):1 + 1
    Xscale = c(Bmin-Blen, Bmin, Bmax, Bmax+Blen, 0)
    iLabs = sprintf('%d', res_m$n)
    ATRANSF = 'identity'
    Fam = 'continuous'
    XLAB = '' 
    if(Outcome %in% BinomOutcomes){
        Xscale = c(log(c(1/3, 0.5, 1, 2, 3)), exp(Bmin), exp(Bmax))
        iLabs = sprintf('%d / %d', res_m$o, res_m$n-res_m$o)
        ATRANSF = 'exp'
        Fam = 'binomial'
        XLAB='Odds Ratio'
    }
    if(Mainrow==2){Ylab = c(Ylab, 0, -2)}
    if(Mainrow==1){Ylab = c(Ylab, 0)}
    forest(mAll, rows = Ylab-1, ylim = c(-4, max(Ylab)+4),
           xlim=c(Xmin, Xmax), at=Xscale,
           ilab = iLabs,
           atransf=eval(parse(text =ATRANSF)),
           ilab.xpos = Bmin-1.6*Blen,
           addfit=F, 
           xlab = XLAB,
           main = sprintf('Sex differences in %s (%s)', Outcome, Fam),
           order=order(factor(res_m$strata, levels=c('PD1', 'PD2', 'PD3', 'PD_all'))))
    text(Xmin, max(Ylab)+2.5, bquote("Cohort"), pos=4, cex=.9)
    if(Outcome %in% BinomOutcomes){
        text(Bmin-2.6*Blen, max(Ylab)+2.5, bquote("Case / Control"), pos=4, cex=.9)
    }else{
        text(Bmin-2.6*Blen, max(Ylab)+2.5, bquote("N"), pos=4, cex=.9)
    }
    text(Xmax, max(Ylab)+2.5, bquote("Mean [95% C.I.]"), pos=2, cex=.9)
  }
  # Add PD meta-analysis results
  op <- par(cex = 1, font=2)
  res_m = res %>% filter(outcome==Outcome, !is.na(beta)) %>%
    filter(strata %in% c('PD_all', 'CTR'))
  N_strata = nrow(res_m)
  if(N_strata==2){
    op <- par(cex = 1, font=4)
    mAll = rma(yi=beta, sei= se, data= res_m, method='REML')
    meta[meta$outcome==Outcome, 3:9] = 
      c(sum(res_m$n), sum(res_m$o), mAll$beta, mAll$se, mAll$pval, mAll$I2, mAll$QEp)
    text(Xmin, -4, pos=4, cex = .9, sprintf('Q-test for main effects = %.2g', mAll$QEp))
    }
    op <- par(cex = 1, font=3)
    text(Xmin, max(Ylab), pos=4, cex=1, bquote('Visualization of sex differences in stratified PD cohorts'))
    
    # Main effects annotations
    op <- par(cex = .9, font=2)
    if(Outcome %in% BinomOutcomes){
        # Interaction effct
        beta1 = mPD[mPD$outcome==Outcome,'beta_slp']
        se1 = mPD[mPD$outcome==Outcome,'se_slp']
        p1 = mPD[mPD$outcome==Outcome,'pval_slp']
        text(Xmin, max(Ylab)+1, pos=4, cex=1, 
             sprintf('Interaction effect between Sex & PD duration:\n OR %.2f [%.2f, %.2f] per year, P = %.2g',
                     exp(beta1), exp(beta1-1.96*se1), exp(beta1+1.96*se1), p1))
        # Main effect in PD
        beta1 = mPD[mPD$outcome==Outcome,'beta_main']
        se1 = mPD[mPD$outcome==Outcome,'se_main']
        p1 = mPD[mPD$outcome==Outcome,'pval_main']
        text(Xmin, 0, pos=4, cex=1, 
             sprintf('Main effect of Sex in PD: OR %.2f [%.2f, %.2f], P = %.2g',
                     exp(beta1), exp(beta1-1.96*se1), exp(beta1+1.96*se1), p1))
        # Main effect in CTR
        if(Mainrow==2){
            beta1 = mCTR[mCTR$outcome==Outcome,'beta_main']
            se1 = mCTR[mCTR$outcome==Outcome,'se_main']
            p1 = mCTR[mCTR$outcome==Outcome,'pval_main']
            text(Xmin, -2, pos=4, cex=1, 
                 sprintf('Main effect of Sex in CTR: OR %.2f [%.2f, %.2f], P = %.2g',
                         exp(beta1), exp(beta1-1.96*se1), exp(beta1+1.96*se1), p1))
            }
        }else{
        # Interaction effct
        beta1 = mPD[mPD$outcome==Outcome,'beta_slp']
        se1 = mPD[mPD$outcome==Outcome,'se_slp']
        p1 = mPD[mPD$outcome==Outcome,'pval_slp']
        text(Xmin, max(Ylab)+1, pos=4, cex=1, 
             sprintf('Interaction effect between Sex & PD duration:\n %.2f [%.2f, %.2f] per year, P = %.2g',
                     beta1, beta1-1.96*se1, beta1+1.96*se1, p1))
        # Main effect in PD
        beta1 = mPD[mPD$outcome==Outcome,'beta_main']
        se1 = mPD[mPD$outcome==Outcome,'se_main']
        p1 = mPD[mPD$outcome==Outcome,'pval_main']
        text(Xmin, 0, pos=4, cex=1, 
             sprintf('Main effect of Sex in PD: OR %.2f [%.2f, %.2f], P = %.2g',
                     beta1, beta1-1.96*se1, beta1+1.96*se1, p1))
        # Main effect in CTR
        if(Mainrow==2){
            beta1 = mCTR[mCTR$outcome==Outcome,'beta_main']
            se1 = mCTR[mCTR$outcome==Outcome,'se_main']
            p1 = mCTR[mCTR$outcome==Outcome,'pval_main']
            if(is.na(beta1))
            text(Xmin, -2, pos=4, cex=1, 
                 sprintf('Main effect of Sex in CTR: OR %.2f [%.2f, %.2f], P = %.2g',
                         beta1, beta1-1.96*se1, beta1+1.96*se1, p1))
        } 
    }
#     dev.off()
}
dev.off()

[1] "UPDRS2"
[1] "PDAQ15_total"
[1] "Dyskinesia"
[1] "Wearing_Off"
[1] "Depression"
[1] "pRBD"
[1] "NMSQ_Awake"
[1] "NMSQ_Sleep"
[1] "NMSQ_Feel"
[1] "NMSQ_Constipation"
[1] "NMSQ_Smell"


png 
  2

In [83]:
left_join(mPD, mCTR, by = 'outcome', suffix = c(".PD", ".CTR")) %>% 
    left_join(., meta[,c('outcome', 'beta', 'se', 'pval', 'QEp')], 
              by = 'outcome',
              suffix = c("", ".Meta")) %>%
    select(-(starts_with('strata'))) %>%
    fwrite(., 'result/_tableFoxInsight.csv')

Warning message:
"Column `outcome` joining character vector and factor, coercing into character vector"


# Table 1

Follow-up should be descirbed in ranges

In [99]:
df = fread('analysisSetFI.csv') %>% 
    mutate(STUDY_NAME = case_when(
        Recruit=='CTR'~paste('Fox Insight', Recruit),
        Recruit %in% c('PD1', 'PD2', 'PD3') ~ 'Fox Insight PD'))
db = db %>% 
    mutate(LEVODOPA = DOPA,
          Female = FEMALE,
          AgeAtBL = AGEatBL,
          WearingOff = Wearing_Off,
          Race = if_else(Race=='', NA_character_, Race),
          EduLevel = case_when(
          YEARSEDUC<9 ~'<9',
          YEARSEDUC<=12 ~ '9-12',
          YEARSEDUC<=16 ~ '13-16',
          YEARSEDUC>16 ~ '>16'))
intersect(names(db), names(df))

[1] "STUDY_NAME"      "ID"              "AGONIST"         "Race"           
 [5] "BMI"             "Wearing_Off"     "Dyskinesia"      "Depression"     
 [9] "pRBD"            "UPDRS2"          "Age"             "DiseaseDuration"
[13] "LEVODOPA"        "Female"          "AgeAtBL"         "WearingOff"     
[17] "EduLevel"

In [100]:
# Variable name conversion
catVars = c('Female', 'LEVODOPA', 'AGONIST', 'WearingOff', 'Dyskinesia')
VarAll = c('Female', 'AgeAtBL', 'Race', 'EduLevel', 'DiseaseDuration', 'LEVODOPA', 'AGONIST', 'WearingOff', 'Dyskinesia')

In [101]:
# Get the replaced obs.
dfb = df %>% 
    arrange(DaysFromIdx) %>%
    group_by(ID) %>%
    filter(!is.na(LEVODOPA) | !is.na(AGONIST) | Recruit=='CTR') %>%
    mutate_at(vars(VarAll), ~na.locf(., na.rm = F)) %>%
    distinct(ID, .keep_all = T)

In [103]:
Table1 = suppressWarnings(
    bind_rows(dfb, db) %>% 
      mutate_at(vars(catVars), as.factor) %>% 
      CreateTableOne(vars=VarAll, 
                          strata = 'STUDY_NAME',
                          test=F) 
) 
print(Table1, quote = FALSE, noSpaces = TRUE, printToggle = FALSE) %>% write.csv('_Table1.csv')

Note: Using an external vector in selections is ambiguous.
i Use `all_of(catVars)` instead of `catVars` to silence this message.
i See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.



In [104]:
db %>% group_by(STUDY_NAME) %>% summarize_at(vars('FUY'), 
                                   list(sum = ~sum(., na.rm = T), 
                                        m = ~mean(., na.rm = T), 
                                        q1 = ~quantile(.,prob = 0.25, na.rm = T),
                                        q2 = ~median(., na.rm = T), 
                                        q3 = ~quantile(.,prob = 0.75, na.rm = T)))

STUDY_NAME,sum,m,q1,q2,q3
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
DATATOP,902.0233,1.133195,0.8049281,1.133470,1.530459
DIGPD,950.5462,2.715846,1.9938398,3.003422,3.999316
HBS,888.2108,1.842761,1.5373032,1.935661,2.141684
NET_PD_LS1,6809.6947,3.993956,2.9705681,3.972621,4.974675
PARKFIT,932.0000,2.000000,2.0000000,2.000000,2.000000
PARKWEST,849.3114,4.692328,4.9472964,5.002053,5.078713
PDBP,1381.5113,2.842616,1.9952088,3.019849,3.690623
PICNICS,409.7769,3.358827,2.0622861,3.471595,4.514031
PPMI,2572.8214,6.305935,5.9575633,6.997947,7.177960


In [105]:
db %>% 
summarize_at(vars('FUY'), 
                                   list(sum = ~sum(., na.rm = T), 
                                        m = ~mean(., na.rm = T), 
                                        q1 = ~quantile(.,prob = 0.25, na.rm = T),
                                        q2 = ~median(., na.rm = T), 
                                        q3 = ~quantile(.,prob = 0.75, na.rm = T)))

sum,m,q1,q2,q3
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
20813.31,3.500388,1.954825,3.081451,4.998631


# Supplementals

In [107]:
# Supplemental Table2
models = c('BL', 'MIX', 'Surv')
models_print = c('Baseline analysis', 'Progression rate analysis', 'Survival analysis')
file_names = 
d = lapply(models, function(x){
        fread(sprintf('result/cohortLevel%s.csv', x)) %>% 
        mutate(ANALYSIS = models_print[which(models==x)]) %>%
        mutate(MODEL = sub('DOPA', 'LEVODOPA', COVS)) %>%
        select(STUDY, ANALYSIS, OUTCOME, MODEL) %>%
        filter(MODEL!='')
    }) %>% bind_rows()

In [108]:
fwrite(d, 'result/SupT_models.csv')

### Further adjustment for dyskinesia-sex associations

In [109]:
d = fread('analysisSet.csv') %>% data.frame

In [110]:
d %>% filter(TSTART==0) %>%
    filter(STUDY_NAME %in% c('NET_PD_LS1', 'PDBP')) %>%
    filter(WGTBL>0) %>%
    mutate(STRATA = paste(STUDY_NAME, FEMALE),
           LDD_WT = LDD/WGTBL) %>%
    CreateTableOne(vars=c('LED', 'LDD', 'WGTBL', 'HTBL', 'LDD_WT'), strata = 'STRATA', data = .)

                    Stratified by STRATA
                     NET_PD_LS1 0    NET_PD_LS1 1    PDBP 0         
  n                    1078             603             244         
  LED (mean (SD))    397.94 (258.06) 360.54 (220.92) 623.28 (378.26)
  LDD (mean (SD))    244.65 (280.21) 214.29 (243.92) 404.76 (333.18)
  WGTBL (mean (SD))   88.55 (15.19)   70.89 (16.16)   87.65 (15.83) 
  HTBL (mean (SD))   177.15 (7.71)   162.72 (6.97)   177.03 (7.22)  
  LDD_WT (mean (SD))   2.85 (3.37)     3.12 (3.57)     4.75 (4.17)  
                    Stratified by STRATA
                     PDBP 1          p      test
  n                     166                     
  LED (mean (SD))    622.35 (573.27) <0.001     
  LDD (mean (SD))    346.80 (363.85) <0.001     
  WGTBL (mean (SD))   69.89 (15.78)  <0.001     
  HTBL (mean (SD))   163.02 (7.00)   <0.001     
  LDD_WT (mean (SD))   5.24 (5.90)   <0.001     

In [113]:
# Analysis
baseCOVs = c('FEMALE', 'Age', 'DiseaseDuration', 'DOPA', 'AGONIST')
newCOVs = c('base', 'BMI', 'WGTBL', 'LDD', 'LED', 'LDDperWGT')
# baseCOVs = c('FEMALE', 'Age', 'DiseaseDuration', 'DOPA', 'AGONIST', 'LDD')
# newCOVs = c('base', 'BMI', 'WGTBL')
d_new = d %>% filter(complete.cases(.[c('BMI', 'WGTBL', 'LDD', 'LED')])) %>%
    mutate(LDDperWGT = LDD/WGTBL)
DF = data.frame(Adjusted=newCOVs, Total_N=NA, Total_E=NA, BETA=NA, SE=NA, P=NA, I2=NA, QEp=NA)
for(addCOV in newCOVs){
    COVs = c(baseCOVs, addCOV)
    if(addCOV=='base'){COVs=baseCOVs}
    result = lapply('Dyskinesia', function(x){coxAnalysis(d_new, x, COVs)})
    result = suppressWarnings(bind_rows(result))
    df_i = result %>% filter(!is.na(BETA))
    k3 = rma(yi=BETA, sei= SE, data=df_i, method = "REML", slab=STUDY)
    DF[DF$Adjusted==addCOV, 2:8]=c(sum(df_i$E), sum(df_i$N), k3$beta, k3$se, k3$pval, k3$I2, k3$QEp)
}


 Dyskinesia done for PPMI , NET_PD_LS1 , PDBP , 
 Dyskinesia done for PPMI , NET_PD_LS1 , PDBP , 
 Dyskinesia done for PPMI , NET_PD_LS1 , PDBP , 
 Dyskinesia done for PPMI , NET_PD_LS1 , PDBP , 
 Dyskinesia done for PPMI , NET_PD_LS1 , PDBP , 
 Dyskinesia done for PPMI , NET_PD_LS1 , PDBP , 

In [114]:
fwrite(DF, 'result/_SupT_adhoc_dyskinesia.csv')

In [115]:
# Forest plot
df_i = result %>% filter(!is.na(BETA)) %>% filter(grepl('WGTBL', COVS)) 
OUTCOMEi='Dyskinesia'
n_all = length(df_i$STUDY)
print(df_i$STUDY, collapse=',')
n_denovo = length(intersect(df_i$STUDY, DENOVOs))
if(n_all>1){
    figout = sprintf('result/%sSurv_supp_Weight.png', OUTCOMEi)
#     png(figout, width=1440, height=1440, res=216)
    if(n_denovo>1){
        k1 = rma(yi=BETA, sei= SE, data= df_i %>% filter(STUDY %in% DENOVOs), method = "REML")
    }else{k1=NULL}
    if(n_all>n_denovo){
        k2 = rma(yi=BETA, sei= SE, data=df_i %>% filter(!(STUDY %in% DENOVOs)), method = "REML", slab=STUDY)
    }else{k2=NULL}
    k3 = rma(yi=BETA, sei= SE, data=df_i, method = "REML", slab=STUDY)
    Ylab = c(n_all + 6 - c(1:n_denovo, (n_denovo+4):(n_all+3)))
    if(n_denovo==0){Ylab = c(n_all + 6 - (n_denovo+4):(n_all+3))}
    
    op <- par(cex = 1, font=1)
    forest(k3, cex=.9, rows = Ylab, ylim = c(-1, max(Ylab) + 4),
         xlim=c(-5, 3), at=log(c(0.25, 0.5, 1, 2, 3, 4)),
         ilab = sprintf('%d / %d', df_i$E, df_i$N),
         ilab.xpos = -2,
         atransf=exp,
         mlab=sprintf('Meta-analysis for all \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                      k3$pval, k3$I2, k3$QEp), 
         addfit=T, xlab = 'Hazard Ratio',
         main = paste(OUTCOMEi, ': sex differences in survival\n(reference - Male)'),
         order=order(factor(df_i$STUDY, levels=STUDYs)))
    text(-5, max(Ylab)+2.5, bquote("Cohort"), pos=4, cex=.9)
    text(-2.6, max(Ylab)+2.5, bquote("Event / N"), pos=4, cex=.8)
    text(3, max(Ylab)+2.5, bquote("HR [95% C.I.]"), pos=2, cex=.9)
    op <- par(cex=.8, font=4)
    if(n_denovo>0){text(-5, max(Ylab)+1, bquote("de novo Cohorts"), pos=4)}
    text(-5,  n_all + 2.8 - n_denovo, "Other Cohorts", pos=4)
    op <- par(cex=0.8, font=3)
    if(n_denovo>1){
        addpoly(k1, atransf=exp, row= n_all + 4.8 - n_denovo, col="grey",
                mlab = sprintf('Meta-analysis for de novo cohorts \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                               k1$pval, k1$I2, k1$QEp))
    }
    if((n_all - n_denovo)>1){
    addpoly(k2, atransf=exp, row= 1.8, col="grey",
            mlab = sprintf('Meta-analysis for the other cohorts \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                           k2$pval, k2$I2, k2$QEp))
    }
    op <- par(cex=1, font=1)
#     dev.off()
}

factor(0)
12 Levels: DATATOP DIGPD HBS NET_PD_LS1 PARKFIT PARKWEST PDBP PICNICS ... UDALL_PENN


### Cognition scale further adjusting for years of education

In [117]:
# YEARSEDUC for cognition
COVs = c('FEMALE', 'Age', 'DiseaseDuration', 'DOPA', 'AGONIST', 'YEARSEDUC')
result = lapply('Cognitive_Impairment', function(x){coxAnalysis(d, x, COVs)})
result = suppressWarnings(bind_rows(result))

# Forest plot
df_i = result %>% filter(!is.na(BETA)) %>% filter(grepl('YEARSEDUC', COVS))
OUTCOMEi='Cognitive_Impairment'
n_all = length(df_i$STUDY)
n_denovo = length(intersect(df_i$STUDY, DENOVOs))
if(n_all>1){
figout = sprintf('result/%sSurv_supp.png', OUTCOMEi)
png(figout, width=1440, height=1440, res=216)
if(n_denovo>1){
k1 = rma(yi=BETA, sei= SE, data= df_i %>% filter(STUDY %in% DENOVOs), method = "REML")
}else{k1=NULL}
if(n_all>n_denovo){
k2 = rma(yi=BETA, sei= SE, data=df_i %>% filter(!(STUDY %in% DENOVOs)), method = "REML", slab=STUDY)
}else{k2=NULL}
k3 = rma(yi=BETA, sei= SE, data=df_i, method = "REML", slab=STUDY)
Ylab = c(n_all + 6 - c(1:n_denovo, (n_denovo+4):(n_all+3)))
if(n_denovo==0){Ylab = c(n_all + 6 - (n_denovo+4):(n_all+3))}
op <- par(cex = 1, font=1)
forest(k3, cex=.9, rows = Ylab, ylim = c(-1, max(Ylab) + 4),
     xlim=c(-5, 3), at=log(c(0.25, 0.5, 1, 2, 3, 4)),
     ilab = sprintf('%d / %d', df_i$E, df_i$N),
     ilab.xpos = -2,
     atransf=exp,
     mlab=sprintf('Meta-analysis for all \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                  k3$pval, k3$I2, k3$QEp), 
     addfit=T, xlab = 'Hazard Ratio',
     main = paste(OUTCOMEi, ': sex differences in survival\n(reference - Male)'),
     order=order(factor(df_i$STUDY, levels=STUDYs)))
text(-5, max(Ylab)+2.5, bquote("Cohort"), pos=4, cex=.9)
text(-2.6, max(Ylab)+2.5, bquote("Event / N"), pos=4, cex=.8)
text(3, max(Ylab)+2.5, bquote("HR [95% C.I.]"), pos=2, cex=.9)
op <- par(cex=.8, font=4)
if(n_denovo>0){text(-5, max(Ylab)+1, bquote("de novo Cohorts"), pos=4)}
text(-5,  n_all + 2.8 - n_denovo, "Other Cohorts", pos=4)
op <- par(cex=0.8, font=3)
if(n_denovo>1){
addpoly(k1, atransf=exp, row= n_all + 4.8 - n_denovo, col="grey",
        mlab = sprintf('Meta-analysis for de novo cohorts \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                       k1$pval, k1$I2, k1$QEp))
}
if((n_all - n_denovo)>1){
addpoly(k2, atransf=exp, row= 1.8, col="grey",
        mlab = sprintf('Meta-analysis for the other cohorts \n(P = %.2g, I_sq = %.1f%%, Q-test = %.2g)', 
                       k2$pval, k2$I2, k2$QEp))
}
op <- par(cex=1, font=1)
}
dev.off()


 Cognitive_Impairment done for PPMI , PreCEPT_PostCEPT , PARKWEST , DATATOP , PICNICS , NET_PD_LS1 , PDBP , HBS , PROPARK , UDALL_PENN , 

png 
  2